In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential, losses, metrics, activations
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def normalize(x):
  # Normalizes to [-1, 1]
  return (x - 127.5) / 127.5

In [ ]:
# Loads data for MNIST

BATCH_SIZE = 64

(x_train, y_train), (x_validation, y_validation) = mnist.load_data()

x_validation, y_validation = x_validation[:500], y_validation[:500]

x_train, x_validation = normalize(x_train), normalize(x_validation)

x_train, x_validation = np.reshape(x_train, (x_train.shape[0], -1)), np.reshape(x_validation, (x_validation.shape[0], -1))

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
         .shuffle(buffer_size = 50_000) \
         .batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# Prinsipper fra DCGAN:
# - Ikke bruk pooling: bruk i stedet konvolusjonelle lag med strides
# - Ikke bruk hidden dense lag (bruk altså bare konvolusjonelle lag i hidden lagene)
# - Bruk batch normalization etter hvert lag, utenom output-laget
#   til generatoren og input-laget til diskriminatoren
# - Bruk ReLU-aktivering for generatoren, med tanh i output-lag (jeg opplevde
#   imidlertid bedre resultater med LeakyReLU også for generatoren)
# - Bruk LeakyReLU-aktivering for diskrimatoren 

In [ ]:
# Definer diskriminator-modellen fra DCGAN-arkitekturen. Denne baserer seg på
# Conv2D-lag. 

discriminator = None

discriminator.compile()
discriminator.summary()


In [ ]:
noise_dim = 20

# Definer generator-modellen fra DCGAN-arkitekturen. Denne baserer seg på
# Conv2DTranspose-lag.

generator = None

generator.compile(loss = losses.BinaryCrossentropy(from_logits = True))
generator.summary()

In [ ]:
def to_image(img):
  return img.reshape([28, 28])

def show_image(img):
  img = to_image(img)
  plt.imshow(img, cmap='gray')
  plt.show()

def generate_noise(dim, batch_size = 1):
  return tf.random.normal([batch_size, dim])

# Generer og viser et bilde fra den utrente generatoren

generated_image = generator(generate_noise(noise_dim)).numpy()

show_image(generated_image)


In [ ]:
# Her kan du bruke samme train_step som i forrige notebook

@tf.function
def train_step(real_images):
  
  D_loss = None
  G_loss = None
  P_real = None
  P_fake = None

  return D_loss, G_loss, P_real, P_fake


In [ ]:
def show_image_grid(images):
  N = images.shape[0]

  fig = plt.figure(figsize=(3, 3))
  fig.set_size_inches(10, 10)

  for i in range(N):
      plt.subplot(3, 3, i+1)
      img = to_image(images[i])
      plt.imshow(img, cmap="gray")
      plt.axis('off')

  plt.show()

In [ ]:
EPOCHS = 20

noise_for_training_visualization = generate_noise(noise_dim, batch_size = 9)

for epoch in range(EPOCHS):

  print("Epoch", epoch),

  for step, (x_train, _) in enumerate(dataset):
    Ld, Lg, Pr, Pf = train_step(x_train)

    if step % 100 == 0:
      print("Step {}. Ld={}, Lg={}, Pr={}, Pf={}".format(step, Ld, Lg, Pr, Pf))

  show_image_grid(generator(noise_for_training_visualization).numpy())

  

In [ ]:
# Denne kan du bruke til å utforske generatoren!

from ipywidgets import interact

p1 = generate_noise(noise_dim)
p2 = generate_noise(noise_dim)
p3 = generate_noise(noise_dim)

@interact(k1=(0, 1.0), k2=(0, 1.0))
def g(k1, k2):
    p = p1 + k1 * (p2 - p1) + k2 * (p3 - p1)
    show_image(generator(p).numpy())